In [2]:
import calendar
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

format_dict = {
    'qty':'{:,}','price':'{:.2f}','amount':'{:,.2f}'
}
cols = 'trade name qty price active'.split()

today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2022, 2, 26), datetime.date(2022, 2, 25))

In [3]:
#yesterday = date(2021, 5, 14)
yesterday

datetime.date(2022, 2, 25)

### Matching check with profits in portmy database

In [17]:
format_dict = {
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
}

In [38]:
sql = '''
SELECT name, buy_target, market, status
FROM stocks
WHERE status IN ('B','O')
ORDER BY market, name'''
stocks = pd.read_sql(sql, conlite)
stocks

,name,buy_target,market,status
0,ASP,3.70,SET,O
1,MAKRO,39.75,SET,B
2,TMT,10.50,SET,B
3,TPIPL,1.70,SET,O
4,BAM,20.50,SET100,O
5,MEGA,40.50,SET100,B
6,RCL,44.50,SET100,B
7,GLOBAL,19.40,SET50,O
8,TU,19.80,SET50,O
9,IMH,21.00,mai,O


In [36]:
sql = '''
SELECT *
FROM dividend
'''
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.dtypes

name                object
q4                 float64
q3                 float64
q2                 float64
q1                 float64
dividend           float64
shares               int64
xdate       datetime64[ns]
paiddate    datetime64[ns]
kind                object
actual               int64
dtype: object

In [37]:
dividend.sort_values(['xdate'],ascending=[True]).style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
22,TCCC,1.8000,0.0000,0.0000,0.0000,1.8000,0,2021-04-22,2021-04-22,,0
25,THANI,0.1700,0.0000,0.1600,0.0000,0.3300,0,2021-04-27,2021-04-27,None,0
50,MST,0.5000,0.0000,0.1000,0.0000,0.6000,0,2021-04-28,2021-04-28,,0
44,SIS,1.0000,0.0000,0.0000,0.0000,1.0000,"4,500",2021-05-14,2021-05-14,,0
41,DRT,0.2200,0.0000,0.2000,0.0000,0.4200,0,2021-05-14,2021-05-14,None,0
46,SC,0.1800,0.0000,0.0000,0.0000,0.1800,0,2021-05-20,2021-05-20,,0
34,PSH,0.6500,0.0000,0.3100,0.0000,0.9600,0,2021-05-21,2021-05-21,None,0
23,DOHOME,0.1000,0.0000,0.0000,0.0000,0.1000,"12,000",2021-05-21,2021-05-21,,0
32,AP,0.4500,0.0000,0.0000,0.0000,0.4500,0,2021-05-27,2021-05-27,,0
55,JMT,0.4100,0.0000,0.4000,0.0000,0.8100,0,2021-09-06,2021-09-06,,0


In [39]:
df_merge = pd.merge(stocks, dividend, on='name', how='outer', indicator=True)
left_mask= df_merge._merge == 'left_only'
df_merge[left_mask]
#no_dividend.sort_values(['xdate'],ascending=[True])

,name,buy_target,market,status,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual,_merge
7,GLOBAL,19.4,SET50,O,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,left_only
9,IMH,21.0,mai,O,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,left_only


### End of Process

### Consensus table

In [4]:
sql = '''
SELECT * FROM consensus ORDER BY name'''
css = pd.read_sql(sql, conlite)
css

,id,name,price,buy,hold,sell,eps_a,eps_b,pe,pbv,yield,consensus_price,status,stock_id
0,2845,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Z,NaN
1,2846,Directory of C:\A8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Z,NaN
2,2844,Volume Serial Number is C496-DC55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Z,NaN
3,2415,AAV,2.50,2.0,4.0,1.0,-0.42,0.05,-6.0,1.6,0.0,2.80,Z,NaN
4,2722,ABM,1.37,1.0,0.0,0.0,0.13,0.16,10.5,1.2,3.6,2.51,Z,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,2753,WP,5.10,1.0,0.0,0.0,0.22,0.36,23.2,2.1,4.9,5.90,Z,NaN
428,2674,WPH,3.26,0.0,0.0,0.0,0.07,0.11,46.6,1.8,0.9,0.00,Z,NaN
429,2648,XO,23.00,3.0,0.0,0.0,1.06,1.16,21.7,7.9,2.8,23.60,Z,NaN
430,2750,ZEN,12.00,3.0,0.0,0.0,-0.29,0.50,-40.9,3.3,0.0,14.57,Z,NaN


In [4]:
sql = '''
SELECT *
FROM profits
ORDER BY name'''
df_prf = pd.read_sql(sql, conmy)
df_prf.shape

(60, 23)

In [5]:
df_merge = pd.merge(df_prf,df_stk,on='name',how='outer',indicator=True)
df_merge.shape

(73, 41)

In [6]:
left_mask= df_merge._merge == 'left_only'
df_merge[left_mask]

,id_x,name,year,quarter,kind,latest_amt_y,previous_amt_y,inc_amt_y,inc_pct_y,latest_amt_q,...,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,_merge
0,2016.0,AP,2021.0,1.0,1.0,5010831.0,2607844.0,2402987.0,92.14,5010831.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
12,2031.0,DRT,2021.0,1.0,1.0,585184.0,527811.0,57373.0,10.87,585184.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
51,2030.0,TCCC,2021.0,1.0,1.0,1702328.0,943810.0,758518.0,80.37,1702328.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [7]:
right_mask= df_merge._merge == 'right_only'
df_merge[right_mask]

,id_x,name,year,quarter,kind,latest_amt_y,previous_amt_y,inc_amt_y,inc_pct_y,latest_amt_q,...,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,_merge
60,NaN,ADVANC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1189.41,0.64,190.00,300.0,-13.0,12.0,600.0,0.0,0.0,right_only
61,NaN,AMATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,276.45,1.46,24.00,3000.0,-13.0,14.0,12000.0,0.0,12.0,right_only
62,NaN,CPNCG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.60,0.39,10.70,10000.0,-4.0,4.0,10000.0,0.0,0.0,right_only
63,NaN,DIF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,195.96,0.24,14.70,10000.0,-5.0,5.0,100000.0,0.0,0.0,right_only
64,NaN,GVREIT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.52,0.22,10.50,10000.0,-9.0,5.0,10000.0,0.0,0.0,right_only
65,NaN,JASIF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,112.30,0.21,10.10,3500.0,-5.0,4.0,100000.0,0.0,0.0,right_only
66,NaN,LH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,459.36,1.09,11.15,6000.0,-6.0,5.0,42000.0,0.0,12.0,right_only
67,NaN,MC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,30.75,0.58,0.00,0.0,-4.0,4.0,0.0,0.0,0.0,right_only
68,NaN,NOBLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,106.66,0.54,8.60,6000.0,-15.0,16.0,18000.0,0.0,0.0,right_only
69,NaN,POPF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.59,0.25,10.90,10000.0,-4.0,4.0,10000.0,0.0,0.0,right_only


In [9]:
both_mask= df_merge._merge == 'both'
df_merge[both_mask]

,id_x,name,year,quarter,kind,latest_amt_y,previous_amt_y,inc_amt_y,inc_pct_y,latest_amt_q,...,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,_merge
1,1984.0,ASIAN,2021.0,1.0,1.0,928093.0,177942.0,750151.0,421.57,928093.0,...,256.10,0.49,12.40,5000.0,-11.0,15.0,25000.0,0.0,12.0,both
2,1935.0,ASK,2021.0,1.0,1.0,958779.0,874555.0,84224.0,9.63,958779.0,...,58.67,0.77,0.00,5000.0,-4.0,4.0,0.0,0.0,0.0,both
3,2007.0,BCH,2021.0,1.0,1.0,1294039.0,1145941.0,148098.0,12.92,1294039.0,...,351.06,0.44,0.00,5000.0,-4.0,4.0,0.0,1.0,0.0,both
4,2020.0,BGC,2021.0,1.0,1.0,537823.0,510234.0,27589.0,5.41,537823.0,...,26.00,0.77,9.90,10000.0,-12.0,14.0,20000.0,0.0,12.0,both
5,2011.0,BKI,2021.0,1.0,1.0,2852925.0,2538062.0,314863.0,12.41,2852925.0,...,19.73,0.13,275.00,300.0,-4.0,4.0,300.0,0.0,0.0,both
6,2013.0,CHG,2021.0,1.0,0.0,941917.0,712417.0,229500.0,32.21,941917.0,...,207.95,0.36,0.00,0.0,-7.0,8.0,0.0,0.0,0.0,both
7,1949.0,COM7,2021.0,1.0,1.0,1768408.0,1254088.0,514320.0,41.01,1768408.0,...,808.08,0.55,38.75,7800.0,-12.0,16.0,0.0,0.0,0.0,both
8,1990.0,CPF,2021.0,1.0,1.0,26856926.0,20287329.0,6569597.0,32.38,26856926.0,...,1358.74,0.93,32.00,10000.0,-9.0,9.0,10000.0,0.0,12.0,both
9,1923.0,DCC,2021.0,1.0,1.0,1711210.0,1033969.0,677241.0,65.50,1711210.0,...,112.76,0.33,2.34,40000.0,-7.0,8.0,40000.0,0.0,12.0,both
10,1907.0,DELTA,2021.0,1.0,1.0,8001920.0,2730385.0,5271535.0,193.07,8001920.0,...,1703.02,1.83,0.00,300.0,-4.0,4.0,0.0,0.0,0.0,both


In [7]:
sql = '''
SELECT * FROM SALES ORDER BY name'''
sales = pd.read_sql(sql, conlite)
sales.shape[0]

4120

In [6]:
sql = '''
DELETE FROM sales
WHERE name = "TCAP"'''
rp = conlite.execute(sql)
rp.rowcount

93